In [2]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import os
import time
import regex as re

## Setando Programação Paralela

In [3]:
MAX_WORKERS = os.cpu_count()
print(MAX_WORKERS)

16


## Buscar Links de todos os Produtos

In [15]:
def search_links(categoria, page,product_links):
    url = f'https://www.drogariaveracruz.com.br/{categoria}/?p={page}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all('a', class_='item-image')

        base_url = "https://www.drogariaveracruz.com.br"

        for link in links:
            href = link.get('href')
            url = f"{base_url}{href}"
            if url not in product_links:
                product_links.append(url)
                print(len(product_links))
                print(page)
        return ''
    else:
        return None


categorias = ['dermocosmeticos','beleza-e-higiene','genericos','home-care','mamae-e-bebe','medicamentos','saude-e-bem-estar']
product_links = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for categoria in categorias:
        page = 1
        while True:
            urls = executor.submit(search_links,categoria,page,product_links).result()
            if urls is not None:
                page += 1
                
        # Tentativa com próxima página
            else: 
                page += 1
                time.sleep(2)
                urls = executor.submit(search_links,categoria,page,product_links).result()
                if urls is not None:
                    pass
                else:
                    break

1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
2
22
2
23
2
24
2
25
2
26
2
27
2
28
2
29
2
30
2
31
2
32
2
33
2
34
2
35
2
36
2
37
2
38
2
39
2
40
2
41
3
42
3
43
3
44
3
45
3
46
3
47
3
48
3
49
3
50
3
51
3
52
3
53
3
54
3
55
3
56
3
57
3
58
3
59
3
60
3
61
4
62
4
63
4
64
4
65
4
66
4
67
4
68
4
69
4
70
4
71
4
72
4
73
4
74
4
75
4
76
4
77
4
78
4
79
4
80
4
81
5
82
5
83
5
84
5
85
5
86
5
87
5
88
5
89
5
90
5
91
5
92
5
93
5
94
5
95
5
96
5
97
5
98
5
99
5
100
5
101
6
102
6
103
6
104
6
105
6
106
6
107
6
108
6
109
6
110
6
111
6
112
6
113
6
114
6
115
6
116
6
117
6
118
6
119
6
120
6
121
7
122
7
123
7
124
7
125
7
126
7
127
7
128
7
129
7
130
7
131
7
132
7
133
7
134
7
135
7
136
7
137
7
138
7
139
7
140
7
141
8
142
8
143
8
144
8
145
8
146
8
147
8
148
8
149
8
150
8
151
8
152
8
153
8
154
8
155
8
156
8
157
8
158
8
159
8
160
8
161
9
162
9
163
9
164
9
165
9
166
9
167
9
168
9
169
9
170
9
171
9
172
9
173
9
174
9
175
9
176
9
177
9
178
9
179
9
180
9
181
10
182
10
183
10
184
10


In [16]:
len(product_links)

11592

In [17]:
df = pd.DataFrame({'Links': product_links})
df.to_csv('links.csv')


## Buscando dados em cada item

In [23]:
def search_ean(soup):
    try:
        ean = soup.find('meta', itemprop='gtin13')['content']
        return ean
    except:
        return None

def search_produto(soup):
    try:
        nome_produto = soup.find('h1', class_='name').text.strip()
        return nome_produto
    except:
        return None

def search_brand(soup):
    try:
        marca = soup.find('a', class_='text-primary font-weight-bold mr-3').text.strip()
        return marca
    except:
        return None
    
def search_farmacia(soup):
    try:
        return 'Drogaria Vera Cruz'
    except:
        return None
    
def search_precos_desconto(soup):
        try:
            preco_sem_desconto = soup.find('p', class_='unit-price').get_text(strip=True)
            preco_sem_desconto = float(preco_sem_desconto.replace('R$', '').replace(',', '.'))
        except:
            return None,None,None
        try:
            preco_com_desconto = soup.find('p', class_='sale-price').strong.get_text(strip=True)
            preco_com_desconto = float(preco_com_desconto.replace('R$', '').replace(',', '.'))
        except:
            preco_com_desconto = preco_sem_desconto
        try:
            porcentagem_desconto = soup.find(class_='descont').get_text(strip=True)
            porcentagem_desconto = abs(float(porcentagem_desconto.replace('%', '')))
        except:
            porcentagem_desconto = 0.0

        return preco_com_desconto,preco_sem_desconto,porcentagem_desconto
    
def search_descricao(soup):
    try:
        descricao = soup.find('div', class_='text-body').get_text(separator='\n').strip()
        return descricao
    except:
        return None

def buscar_info_products(url,ean,produto,marca,farmacia,preco_com_desconto,preco_sem_desconto,desconto,descricao,i):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url,headers=headers)
    print(i)
    
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        ean.append(search_ean(soup))
        produto.append(search_produto(soup))
        marca.append(search_brand(soup))
        farmacia.append(search_farmacia(soup))
        descricao.append(search_descricao(soup))
        product_preco_desconto,product_preco_sem_desconto,product_desconto = search_precos_desconto(soup)
        preco_com_desconto.append(product_preco_desconto)
        preco_sem_desconto.append(product_preco_sem_desconto)
        desconto.append(product_desconto)
    else:
        ean.append(None)
        produto.append(None)
        marca.append(None)
        farmacia.append(None)
        preco_com_desconto.append(None)
        preco_sem_desconto.append(None)
        desconto.append(None)
        descricao.append(None)



In [24]:
ean = []
produto = []
marca = []
farmacia = []
preco_com_desconto = []
preco_sem_desconto = []
desconto = []
descricao = []
i = 1
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submeter cada busca em paralelo
    for url in product_links:
        executor.submit(buscar_info_products, url, ean, produto, marca, farmacia, preco_com_desconto, preco_sem_desconto, desconto, descricao,i)
        i += 1

2
1
3
7
4
9
5
14
8
12
610
18

16
13
11
17
15
19
20
21
29
26
27
23
24
31
34
22
25
35
28
33
30
32
36
41
39
37
44
42
46
38
45
43
40
47
50
52
51
48
53
49
54
56
57
55
59
66
71
60
58
6361

68
65
64
67
62
69
72
70
74
75
77
88
76
85
78
83
86
87
79
73
80
81
84
82
89
90
92
96
91
101
102
94
104
93
98
95
100
97
99
105
106
103
107
108
109
111
115
117
116
120
121
112
119
113
118
114
110
123
122
125
124
130
128
126
129
131
127
138136

135
132
134
137
139
133
140
144
141
142
146
143
148
149
147
145
150
153
156
155
151
152
154
159
157
161
158
160
163
165
167
164
166
170
162
171
168
172
169
173
174
179
177
180
176
175
181
186
182
184
183
178
188
189
187
185
190
191
193
194
192
198
195
197
201
200
199
203
204
202
196
206
205
211
210
212
208
213
209
207
218
221
220
214
216
219
217
215
222
225
223
224
235
229
228
236
226
230
232
227
231233

234
237
240
239
238
241
243
242
245
252
248
249
247
246
255
251
254
244
256250
253

258
259
257
262
260261

266
263
264
265
269
270
275
268
271
267
274
273
272
276
278


In [25]:
print(len(ean))
print(len(produto))
print(len(marca))
print(len(farmacia))
print(len(preco_com_desconto))
print(len(preco_sem_desconto))
print(len(desconto))
print(len(descricao))

11591
11591
11591
11591
11591
11591
11591
11591


In [26]:
df = pd.DataFrame({
    'ean': ean,
    'produto': produto,
    'marca': marca,
    'farmacia': farmacia,
    'preco com desconto': preco_com_desconto,
    'preco sem desconto': preco_sem_desconto,
    '% desconto': desconto,
    'descricao': descricao
})

In [27]:
df.head(50)

,ean,produto,marca,farmacia,preco com desconto,preco sem desconto,% desconto,descricao
0,7897947601864,Hidratante Labial Carmed Fini Dentaduras Incol...,Cimed,Drogaria Vera Cruz,17.90,22.99,22.0,Carmed Fini Dentaduras é um hidratante labial ...
1,7897947601857,Hidratante Labial Carmed Fini Beijos com Cor E...,Cimed,Drogaria Vera Cruz,36.80,36.80,0.0,O que é?\nO Hidratante Labial Carmed Fini Beij...
2,7897947601840,Hidratante Labial Carmed Fini Bananas Incolor ...,Cimed,Drogaria Vera Cruz,17.90,22.99,22.0,O que é?\nHidratante Labial Carmed Fini Banana...
3,7899785200780,Creme Hidratante Skinphil Pele Seca e Extrasse...,Cimed,Drogaria Vera Cruz,49.32,64.90,24.0,O que é?\nO Creme Hidratante Skinphil é um pro...
4,7896112421979,Loção Antiperspirante MP Dry 60ml,Biolab Sanus,Drogaria Vera Cruz,96.92,108.90,11.0,O que é?\nA MP Dry Loção Antiperspirante ofere...
5,7899706254403,Shampoo Antiqueda Vichy Dercos Energy+ 400g,Vichy,Drogaria Vera Cruz,129.99,129.99,0.0,O que é?\nDercos Energy+ é uma linha de produt...
6,7897129304552,Sérum Anti-Idade TheraSkin Euryale C 50g,Theraskin,Drogaria Vera Cruz,135.90,135.90,0.0,EURYALE®C é um sérum com 10% de Vitamina C pur...
7,3337875686471,Protetor Solar Facial La Roche-Posay Anthelios...,La Roche-posay,Drogaria Vera Cruz,70.96,87.60,19.0,"Protetor solar sem cor com ação anti-idade, te..."
8,7897947616882,Hidratante Labial Carmed Maça do Amor 10g,Cimed,Drogaria Vera Cruz,101.11,114.90,12.0,Hidratante Labial Carmed Maçã do Amor possui a...
9,3282770075526,Shampoo Antiqueda Ducray Anaphase+ 400ml,Ducray,Drogaria Vera Cruz,20.39,23.99,15.0,Shampoo Ducray Anaphase+ é o complemento ideal...


In [29]:
df.to_csv('info_produtos_vera_cruz.csv')